# **PHASE 2: ADVANCED SVD**

## Import các thư viện cần thiết

In [52]:
import os, math, random, gc, time, pickle
from collections import Counter, defaultdict

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from scipy import sparse

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, normalize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Ridge

from surprise import SVD, SVDpp, Dataset, Reader, accuracy
from IPython.display import HTML, display



## Chuẩn bị dữ liệu

### 1. Config

In [2]:
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

DATA_DIR = "/kaggle/input/ml-25m"
Z_THRESHOLD = 0.0    # z-score >= 0 ⇒ user "thích" phim
RATING_MIN, RATING_MAX = 0.5, 5.0
TEST_SIZE, VAL_SIZE = 0.10, 0.10
N_FEATURES_FS = 300     # Số lượng features sẽ lấy khi feature selection

### 2. Load data

In [50]:
# Read file csv
ratings = pd.read_csv(f"{DATA_DIR}/ratings.csv")
movies = pd.read_csv(f"{DATA_DIR}/movies.csv")
tags = pd.read_csv(f"{DATA_DIR}/tags.csv")
genome_scores = pd.read_csv(f"{DATA_DIR}/genome-scores.csv")
genome_tags = pd.read_csv(f"{DATA_DIR}/genome-tags.csv")
links = pd.read_csv(f"{DATA_DIR}/links.csv")

### 3. Merge data

In [4]:
# Merge để có title ngay trong ratings
df = pd.merge(ratings, movies, on='movieId')

total_movies  = movies["movieId"].nunique()          # tổng số phim trong bảng movies
rated_movies  = ratings["movieId"].nunique()         # phim đã có ít nhất 1 rating
tagged_movies = tags["movieId"].nunique()            # phim xuất hiện trong bảng tags

print(f"Original dataset : {len(df):,} ratings | "
      f"{df['userId'].nunique():,} users | "
      f"{df['movieId'].nunique():,} movies (after merge)")

print(f"Total movies      : {total_movies:,}")
print(f"Rated movies      : {rated_movies:,}")       
print(f"Movies in tags    : {tagged_movies:,}")
display(df.head())

Original dataset : 25,000,095 ratings | 162,541 users | 59,047 movies (after merge)
Total movies      : 62,423
Rated movies      : 59,047
Movies in tags    : 45,251


,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance


### 4. Encode

In [5]:
# Encode userId và movieId thành chỉ số liên tục từ 0 để cho indexing ma trận
user_encoder = LabelEncoder()
user_encoder.fit(df['userId'])
df['user_idx'] = user_encoder.transform(df['userId'])


movie_encoder = LabelEncoder()
movie_encoder.fit(df['movieId'])
df['movie_idx'] = movie_encoder.transform(df['movieId'])

n_users  = df["user_idx"].nunique()
n_items  = df["movie_idx"].nunique()
print(f"Dataset: {len(df):,} ratings | {n_users:,} users | {n_items:,} movies")

Dataset: 25,000,095 ratings | 162,541 users | 59,047 movies


### 5. Split data

In [6]:
output_dir = "/kaggle/working"

In [7]:
# Nếu chưa có thì tạo folder lưu file chia train/test/val
split_dir = os.path.join(output_dir, "splits")
os.makedirs(split_dir, exist_ok=True)

train_path = os.path.join(split_dir, "train.csv")
val_path = os.path.join(split_dir, "val.csv")
test_path = os.path.join(split_dir, "test.csv")

In [8]:
# Nếu đã có file thì chỉ cần load không cần split 
if os.path.exists(train_path) and os.path.exists(val_path) and os.path.exists(test_path):
    print("Loading cached train/val/test splits...")
    train_df = pd.read_csv(train_path)
    val_df = pd.read_csv(val_path)
    test_df = pd.read_csv(test_path)
    
    train_df['user_idx'] = user_encoder.transform(train_df['userId'])
    val_df['user_idx'] = user_encoder.transform(val_df['userId'])
    test_df['user_idx'] = user_encoder.transform(test_df['userId'])
else:
    print("Splitting train/val/test...")
    def train_val_test_split_func(data_frame, test_size=TEST_SIZE, val_size=VAL_SIZE, col="user_idx", seed=SEED):
        train_val, test = train_test_split(
            data_frame, 
            test_size=test_size, 
            random_state=seed, 
            stratify=data_frame[col]
        )
        train, val = train_test_split(
            train_val, 
            test_size=val_size, 
            random_state=seed, 
            stratify=train_val[col]
        )
        return train.reset_index(drop=True), val.reset_index(drop=True), test.reset_index(drop=True)

    train_df, val_df, test_df = train_val_test_split_func(df)
    
    # Lưu lại file để lần sau load nhanh hơn
    train_df.to_csv(train_path, index=False)
    val_df.to_csv(val_path, index=False)
    test_df.to_csv(test_path, index=False)

print("Split sizes:", {k: len(v) for k, v in zip(["train", "val", "test"], [train_df, val_df, test_df])})


Splitting train/val/test...
Split sizes: {'train': 20250076, 'val': 2250009, 'test': 2500010}


## **Thuật toán FunkSVD**

### 1. Cài đặt thuật toán

- **Mục tiêu:** Xấp xỉ ma trận đánh giá thưa $$ R \in \mathbb{R}^{n_{\text{users}} \times n_{\text{items}}} $$ bằng tích của hai ma trận yếu tố ẩn  
  $$ P \in \mathbb{R}^{n_{\text{users}} \times k}, \quad Q \in \mathbb{R}^{n_{\text{items}} \times k}, \quad R \approx PQ^T $$

- **Dự đoán:** Điểm đánh giá dự đoán cho người dùng $ u $ và phim $ i $:

  $$
  \hat{r}_{ui} = \mu + b_u + b_i + P_u \cdot Q_i
  $$

  Trong đó:

  - $\mu $: Trung bình toàn bộ đánh giá (global bias).
  - $ b_u $: Bias của người dùng $u$ (xu hướng đánh giá cao/thấp).
  - $ b_i $: Bias của phim $ i $ (xu hướng được đánh giá cao/thấp).
  - $ P_u \cdot Q_i $: Tích vô hướng của vector yếu tố ẩn (kích thước $ k $).

- **Hàm mất mát:** Tối thiểu hóa sai số bình phương với điều chuẩn riêng cho bias và yếu tố ẩn:

  $
  L = \sum_{(u,i) \in \text{known ratings}} (r_{ui} - \hat{r}_{ui})^2 + \lambda_{bu,bi} \left( \sum_u b_u^2 + \sum_i b_i^2 \right) + \lambda_{pq} \left( \sum_u \| P_u \|^2 + \sum_i \| Q_i \|^2 \right)
  $

  Trong đó:

  - $ (r_{ui} - \hat{r}_{ui})^2 $: Sai số bình phương.
  - $ \lambda_{bu,bi} $: Hệ số điều chuẩn cho bias (`reg_bu_bi`).
  - $ \lambda_{pq} $: Hệ số điều chuẩn cho yếu tố ẩn (`reg_pq`).
  - $ b_u^2, b_i^2, \|P_u\|^2, \|Q_i\|^2 $: Phạt các giá trị lớn để tránh overfitting.

- **Tối ưu hóa:** Sử dụng SGD để cập nhật $ \mu, b_u, b_i, P_u, Q_i $ từng mẫu dữ liệu một cách ngẫu nhiên, với learning rate giảm dần (decay).


In [9]:
# Hàm df_to_numpy() dùng để chuyển DataFrame sang NumPy
def df_to_numpy(df_):
    arr = df_[["user_idx", "movie_idx", "rating"]].to_numpy()
    return arr[:, 0].astype(np.int32), arr[:, 1].astype(np.int32), arr[:, 2].astype(np.float32)

In [10]:
class SurpriseFunkSVD:
    def __init__(
        self,
        n_factors=50,
        lr_all=0.007,
        reg_all=0.02,
        n_epochs=20,
        verbose=False,
        rating_scale=(0.5, 5.0)
    ):
        self.model = SVD(n_factors=n_factors, lr_all=lr_all, reg_all=reg_all,
                         n_epochs=n_epochs, verbose=verbose)
        self.rating_scale = rating_scale

    def fit(self, train_df):
        reader = Reader(rating_scale=self.rating_scale)
        data = Dataset.load_from_df(train_df[['user_idx', 'movie_idx', 'rating']], reader)
        trainset = data.build_full_trainset()
        self.model.fit(trainset)
        return self

    def predict(self, u_idx: int, i_idx: int) -> float:
        est = self.model.predict(uid=u_idx, iid=i_idx).est
        return float(np.clip(est, *self.rating_scale))

    def predict_batch(self, users, items, disable_tqdm=False):
        preds = []
        for u, i in tqdm(zip(users, items), total=len(users), desc="Predicting", disable=disable_tqdm):
            preds.append(self.predict(u, i))
        return np.array(preds)


    def recommend_for_nan(self, df):
        df_nan = df[df["rating"].isna()].copy()
        predictions = self.predict_batch(df_nan["user_idx"].values, df_nan["movie_idx"].values)
        df_nan["predicted_rating"] = predictions
        return df_nan

    def save(self, file_path: str):
        import joblib
        joblib.dump(self.model, file_path)

    def load(self, file_path: str):
        import joblib
        self.model = joblib.load(file_path)
        return self

In [11]:
def evaluate(df, model):
    u, i, r = df["user_idx"], df["movie_idx"], df["rating"]
    p = model.predict_batch(u, i)
    return np.sqrt(mean_squared_error(r, p)), mean_absolute_error(r, p)

# tr_rmse, tr_mae = evaluate(train_df, model_funksvd)
# vl_rmse, vl_mae = evaluate(val_df, model_funksvd)
# ts_rmse, ts_mae = evaluate(test_df, model_funksvd)

# print(
#     f"\nRMSE  | MAE\n"
#     f"Train: {tr_rmse:.4f} | {tr_mae:.4f}\n"
#     f"Val  : {vl_rmse:.4f} | {vl_mae:.4f}\n"
#     f"Test : {ts_rmse:.4f} | {ts_mae:.4f}"
# )


## **Hybrid Model**

### **A. Content-Based**

##### **Sử dụng 2 phương pháp:**

- **Cosine similarity:** Tính độ tương đồng giữa hồ sơ người dùng (dựa trên các phim họ thích) và đặc trưng của phim (genome tags và thể loại) để gợi ý phim tương tự.

- **Ridge Regression:** Học một mô hình tuyến tính cho mỗi người dùng để dự đoán điểm đánh giá dựa trên đặc trưng phim, từ đó gợi ý phim có điểm dự đoán cao.

#### 1. Save/Load 

In [12]:
# Folder cache lưu hồ sơ người dùng cho content-based
PROFILE_CACHE_DIR = "./cache_profiles"
os.makedirs(PROFILE_CACHE_DIR, exist_ok=True)

# Hàm save user profiles 
def save_profiles(profiles_dict, method_name):
    file_path = os.path.join('/kaggle/working', f"profiles_{method_name}.pkl")
    with open(file_path, "wb") as f:
        pickle.dump(profiles_dict, f)

# Hàm load user profiles 
def load_profiles(method_name):
    file_path = os.path.join('/kaggle/input/ml-25m/', f"profiles_{method_name}.pkl")
    if os.path.exists(file_path):
        with open(file_path, "rb") as f:
            return pickle.load(f)
    return None

In [13]:
GENOME_CACHE_PATH = os.path.join('/kaggle/input/ml-25m/', "genome_csr_best.pkl") 

# Hàm savema trận genome CSR 
def save_genome_csr(csr):  
    with open(GENOME_CACHE_PATH, "wb") as f:
        pickle.dump(csr, f)

# Hàm load ma trận genome CSR 
def load_genome_csr():  
    if os.path.exists(GENOME_CACHE_PATH):
        with open(GENOME_CACHE_PATH, "rb") as f:
            return pickle.load(f)
    return None

#### 2. Per‑user z‑score (remove bias)

Mỗi người dùng có thang chấm điểm khác nhau. Ví dụ, một người có thể chấm 4/5 là "rất thích", trong khi người khác chấm 4/5 là "bình thường". Chuẩn hóa z-score giúp so sánh đánh giá giữa các người dùng một cách công bằng.

In [14]:
# Tính mean và std rating cho từng user trong tập train (để scale)
train_user_stats = train_df.groupby("userId")["rating"].agg(
        ["mean", "std"]).rename(columns={"mean": "mu", "std": "sigma"})

**Z-score:** `z = (x - mu) / sigma`, trong đó `x` là đánh giá, `mu` là trung bình đánh giá của người dùng, `sigma` là độ lệch chuẩn.

- Z-score biểu thị đánh giá lệch bao nhiêu so với trung bình của người dùng, chuẩn hóa về đơn vị độ lệch chuẩn.

- Nếu `sigma=0` (người dùng chỉ chấm một giá trị duy nhất), phép chia sẽ gây lỗi (chia cho 0). Thay bằng 1e-6 để tránh lỗi và giữ z-score hợp lý.

In [15]:
# Hàm thêm cột z-score vào DataFrame 
def add_z_scores(df, user_stats):
    df_with_stats = df.merge(user_stats, on="userId", how="left")

    ratings_mean = train_df["rating"].mean()
    ratings_std = train_df["rating"].std()

    # Nếu user chưa từng rating, fill bằng mean của all data
    df_with_stats["mu"] = df_with_stats["mu"].fillna(ratings_mean)
    df_with_stats["sigma"] = df_with_stats["sigma"].fillna(ratings_std)
    
    # Tính z-score = (x - mu) / sigma (tránh chia 0)
    df_with_stats["rating_z"] = (df_with_stats["rating"] - df_with_stats["mu"]) / df_with_stats["sigma"].replace(0, 1e-6)
    
    return df_with_stats

# Apply cho train rồi thực hiện trên cả tập val và test
train_df = add_z_scores(train_df, train_user_stats)
val_df = add_z_scores(val_df, train_user_stats)
test_df = add_z_scores(test_df, train_user_stats)

#### 3. Build genome CSR matrix

In [16]:
# Gộp genome scores và genome tags lại với nhau (theo tagId)
merged_genome = pd.merge(genome_scores, genome_tags, on='tagId', how='left')

# Pivot để mỗi dòng là movieId, mỗi cột là tag, giá trị là relevance
genome_matrix = merged_genome.pivot(index='movieId', columns='tag', values='relevance').fillna(0)

#### 4. Genres Vectorization (TF-IDF)

In [17]:
# Vector hóa genres (thể loại phim) dùng TF-IDF cho từng phim
tfidf = TfidfVectorizer(token_pattern=r'[^|]+')
genres_tfidf = tfidf.fit_transform(movies['genres'])
genres_df = pd.DataFrame(
    genres_tfidf.toarray(), 
    columns=[f'genre:{g}' for g in tfidf.get_feature_names_out()],
    index=movies['movieId']
)

#### 5. Xử lí, gộp ma trận và scale

In [18]:
all_movies = pd.Index(sorted(set(genome_matrix.index).union(genres_df.index)))

# Fill 0 cho phim thiếu genome
missing_genome = all_movies.difference(genome_matrix.index)
if len(missing_genome):
    zero_genome = pd.DataFrame(0, index=missing_genome, columns=genome_matrix.columns)
    genome_matrix = pd.concat([genome_matrix, zero_genome])

# Fill 0 cho phim thiếu genres
missing_genre = all_movies.difference(genres_df.index)
if len(missing_genre):
    zero_genre = pd.DataFrame(0, index=missing_genre, columns=genres_df.columns)
    genres_df = pd.concat([genres_df, zero_genre])

In [19]:
# chỉnh lại index giống nhau khi gộp 2 ma trận
genome_matrix = genome_matrix.loc[all_movies]
genres_df = genres_df.loc[all_movies]

In [20]:
# Ghép hai ma trận -> describe_matrix
describe_matrix = pd.concat([genome_matrix, genres_df], axis=1).fillna(0)
print("Describe matrix:", describe_matrix.shape)    

Describe matrix: (62423, 1148)


In [21]:
# Scale lại ma trận cho cùng range
scaler = StandardScaler(with_mean=False)
describe_matrix_scaled = scaler.fit_transform(describe_matrix) 
print(describe_matrix_scaled)

[[0.7411119  0.68198834 1.0122178  ... 0.         0.         0.        ]
 [1.06333447 1.16296959 1.01626667 ... 0.         0.         0.        ]
 [1.2051124  1.59369907 0.47371793 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


#### 6. Feature Selection

In [22]:
# Tính mean rating từng phim trên tập train, chuyển rating thành class để chọn đặc trưng phân loại
movie_avg_train = train_df.groupby("movieId")["rating"].mean()

def rating_to_class(rating):
    if rating <= 2.5:
        return 0  # rating thấp
    elif rating <= 4.0:
        return 1  # rating trung bình
    else:
        return 2  # rating cao

# Chỉ lấy những phim vừa có đặc trưng, vừa có rating trung bình
train_movies_fs = [m for m in movie_avg_train.index if m in describe_matrix.index]

In [ ]:
X_fs = describe_matrix_scaled[[describe_matrix.index.get_loc(m) for m in train_movies_fs]]
y_fs = movie_avg_train.loc[train_movies_fs].apply(rating_to_class).values
k = min(N_FEATURES_FS, X_fs.shape[1])

# Tạo các ma trận đặc trưng theo 4 phương pháp: chi2, entropy, gini, std deviation
feature_matrices = {}

# 1. Chi-squared
sel_chi = SelectKBest(chi2, k=k).fit(X_fs, y_fs)
feature_matrices["chi2"] = sel_chi.transform(describe_matrix_scaled)

if sparse.issparse(X_fs):        
    X_fs_dense = X_fs.toarray()
else:
    X_fs_dense = X_fs

# 2. Information Gain (Entropy)
tree_e = DecisionTreeClassifier(criterion="entropy", random_state=SEED).fit(X_fs_dense, y_fs)
idx_e = np.argsort(tree_e.feature_importances_)[-k:]
feature_matrices["entropy"] = describe_matrix_scaled[:, idx_e]

# 3. Gini Index
tree_g = DecisionTreeClassifier(criterion="gini", random_state=SEED).fit(X_fs_dense, y_fs)
idx_g = np.argsort(tree_g.feature_importances_)[-k:]
feature_matrices["gini"] = describe_matrix_scaled[:, idx_g]

# 4. Normalized Deviation
std_col = np.std(X_fs_dense, axis=0)
idx_std = np.argsort(std_col)[-k:]
feature_matrices["deviation"] = describe_matrix_scaled[:, idx_std]

#### **Cosine Similarity**

Làm thế nào để biểu diễn sở thích của người dùng?

- Hồ sơ người dùng là một vector tổng hợp các tag genome của những phim họ thích (dựa trên `rating_z ≥ Z_THRESHOLD`). Vector này được tính bằng trung bình có trọng số của các vector phim.

In [23]:
# Tạo mapping từ movieId sang dòng trong describe_matrix
movieId2row = {mid: i for i, mid in enumerate(describe_matrix.index.values)}
row2movieId = describe_matrix.index.values

In [24]:
# Hàm build user profile cho phương pháp cosine similarity
def build_user_profiles_sim(ratings_df, rating_col="rating_z", threshold=Z_THRESHOLD, eps=1e-8):
    profiles = {}
    good = ratings_df.loc[ratings_df[rating_col] >= threshold, ["userId", "movieId", rating_col]]
    
    for uid, grp in tqdm(good.groupby("userId"), desc="profiles", unit="user"):
        rows = [movieId2row[m] for m in grp.movieId if m in movieId2row]
        if not rows:
            continue

        w = grp[rating_col].values[:, None]
        
        if w.shape[0] == len(rows):
            w_sum = w.sum()

            if w_sum < eps:
                prof_dense = genome_csr[rows].mean(axis=0)
            else:
                prof_dense = (genome_csr[rows].multiply(w)).sum(axis=0) / w_sum

            prof_dense = np.nan_to_num(np.asarray(prof_dense).ravel())
            profiles[uid] = sparse.csr_matrix(prof_dense)

    return profiles

# Hàm đánh giá cosine similarity
def eval_cosine(df_eval, profiles):
    df_eval = df_eval[df_eval.movieId.isin(movieId2row)]
    y_true, y_pred = [], []
    iterator = zip(df_eval.userId.values, df_eval.movieId.values, df_eval.rating.values)
    for u, m, r in tqdm(iterator, total=len(df_eval), desc="eval_cosine", unit="rec"):
        y_true.append(r)
        if u in profiles:
            sim = cosine_similarity(profiles[u], genome_csr[movieId2row[m]])[0, 0]
            pred = sim * (RATING_MAX - RATING_MIN) + RATING_MIN
        else:
            pred = RATING_MIN
        y_pred.append(pred)
    return math.sqrt(mean_squared_error(y_true, y_pred)), mean_absolute_error(y_true, y_pred)

# Hàm thử nghiệm cosine cho từng phương pháp chọn đặc trưng
def test_cosine_method(mat, method_name):
    global genome_csr, profiles
    genome_csr = normalize(sparse.csr_matrix(mat), axis=1, copy=False)

    profiles = load_profiles(method_name)
    if profiles is None:
        profiles = build_user_profiles_sim(train_df)
        save_profiles(profiles, method_name)

    rmse_val, mae_val = eval_cosine(val_df,  profiles)
    rmse_tst, mae_tst = eval_cosine(test_df, profiles)
    return rmse_val, mae_val, rmse_tst, mae_tst

In [25]:
def predict_rating_cosine(user_id, movie_ids, profiles, movieId2row_dict, genome_csr_matrix, normalize_rating=True):

    if user_id not in profiles:
        return np.array([]), []
    
    user_profile = profiles[user_id]
    valid_movie_ids = [m for m in movie_ids if m in movieId2row_dict]
    
    if not valid_movie_ids:
        return np.array([]), []
    
    # Lấy row indices cho các phim hợp lệ
    movie_rows = [movieId2row_dict[m] for m in valid_movie_ids]
    
    # Tính cosine similarity giữa user profile và từng phim
    similarities = []
    for row in movie_rows:
        sim = cosine_similarity(user_profile, genome_csr_matrix[row])[0, 0]
        similarities.append(sim)
    
    # Chuyển similarity thành predicted rating
    pred_ratings = np.array(similarities) * (RATING_MAX - RATING_MIN) + RATING_MIN
    
    if normalize_rating:
        pred_ratings = np.clip(pred_ratings, RATING_MIN, RATING_MAX)
    
    return pred_ratings

#### **Ridge Regression**

In [26]:
# Hàm lấy danh sách phim và rating của 1 user
def get_items_rated_by_user(ratings_df, user_id):
    user_df = ratings_df[ratings_df['userId'] == user_id]
    movies_list = user_df['movieId'].values
    ratings_list = user_df['rating'].values
    return movies_list, ratings_list

# Hàm train mô hình Ridge Regression cho từng user (build user profiles với phương pháp Ridge Regression)
def build_user_profiles_ridge(ratings_df, descriptions_df, min_ratings_for_grid=50, verbose=True):
    start = time.time()
    profiles = {}
    users_ids = ratings_df['userId'].unique()

    if verbose:
        print(f"Training {len(users_ids)} users...")

    param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

    for uid in tqdm(users_ids) if verbose else users_ids:
        uid_movies, uid_ratings = get_items_rated_by_user(ratings_df, uid)

        valid_movies = [m for m in uid_movies if m in descriptions_df.index]
        if not valid_movies:
            continue

        X = descriptions_df.loc[valid_movies].values
        y = uid_ratings[:len(valid_movies)] 

        if len(y) >= min_ratings_for_grid:
            try:
                grid = GridSearchCV(
                    Ridge(), 
                    param_grid, 
                    scoring='neg_mean_squared_error',
                    cv=min(5, len(y)), 
                    n_jobs=-1
                )
                grid.fit(X, y)
                profiles[uid] = grid.best_estimator_
            except:
                fallback_model = Ridge(alpha=1.0)
                fallback_model.fit(X, y)
                profiles[uid] = fallback_model
        else:
            fallback_model = Ridge(alpha=1.0)
            fallback_model.fit(X, y)
            profiles[uid] = fallback_model

    if verbose:
        print(f'Training time: {(time.time() - start):.4f}s')
    return profiles

# Hàm dự đoán rating cho 1 user với nhiều phim (Ridge)
def predict_rating_ridge(user_id, movie_ids, profiles, descriptions_df, normalize_rating=True):
    if user_id not in profiles:
        return np.array([])

    model = profiles[user_id]
    valid_movie_ids = [m for m in movie_ids if m in descriptions_df.index]
    
    if len(valid_movie_ids) == 0:  # <-- Sửa chỗ này
        return np.array([])

    X = descriptions_df.loc[valid_movie_ids].values
    pred_rating = model.predict(X)

    if normalize_rating:
        pred_rating = np.clip(pred_rating, RATING_MIN, RATING_MAX)
    
    return pred_rating




# Hàm evaluate cho ridge
def eval_ridge(ratings_df, profiles, descriptions, user_id=None):
    if user_id is None:
        users_ids = ratings_df['userId'].unique()
        y_true, y_pred = [], []

        for uid in tqdm(users_ids):
            test_movies, true_ratings = get_items_rated_by_user(ratings_df, uid)
            pred_ratings = predict_rating_ridge(uid, test_movies, profiles, descriptions)
            if len(pred_ratings) == len(true_ratings):
                y_true.extend(true_ratings)
                y_pred.extend(pred_ratings)
    else:
        test_movies, true_ratings = get_items_rated_by_user(ratings_df, user_id)
        pred_ratings = predict_rating_ridge(user_id, test_movies, profiles, descriptions, ratings_df)
        y_true, y_pred = true_ratings, pred_ratings

    return math.sqrt(mean_squared_error(y_true, y_pred)), mean_absolute_error(y_true, y_pred)


#### **Main Excutor**

In [ ]:
results = {}
for name, mat in feature_matrices.items():
    print(f"Đánh giá feature matrix: {name}")

    # Cosine: chuẩn hóa ma trận, build profile, đánh giá trên val/test
    genome_csr = normalize(sparse.csr_matrix(mat), axis=1, copy=False)
    profiles_cosine = build_user_profiles_sim(train_df) 
    rmse_val_cosine, mae_val_cosine = eval_cosine(val_df, profiles_cosine)
    rmse_test_cosine, mae_test_cosine = eval_cosine(test_df, profiles_cosine)

    # # Ridge: build DataFrame từ sparse matrix, build profile, đánh giá 
    # desc_df = pd.DataFrame(
    #     data=normalize(sparse.csr_matrix(mat), axis=1, copy=False),     
    #     index=describe_matrix.index
    # )

    # profiles_ridge = build_user_profiles_ridge(train_df, desc_df)
    # rmse_val_ridge, mae_val_ridge = eval_ridge(val_df, profiles_ridge, desc_df)
    # rmse_test_ridge, mae_test_ridge = eval_ridge(test_df, profiles_ridge, desc_df)

    results[name] = {
    'cosine': (rmse_val_cosine, mae_val_cosine, rmse_test_cosine, mae_test_cosine)
    # 'ridge' : (rmse_val_ridge, mae_val_ridge, rmse_test_ridge, mae_test_ridge)
    }

    print(f"Cosine - Val RMSE {rmse_val_cosine:.4f} | Test RMSE {rmse_test_cosine:.4f}")
    # print(f"Ridge - Val RMSE {rmse_val_ridge:.4f} | Test RMSE {rmse_test_ridge:.4f}")

In [ ]:
# Tìm ra phương pháp chọn đặc trưng tốt nhất cho cosine và ridge
best_method_cosine = min(results, key=lambda n: results[n]['cosine'][0])   
# best_method_ridge  = min(results, key=lambda n: results[n]['ridge'][0])    

print(f"Best Cosine feature matrix: {best_method_cosine}")
# print(f"Best Ridge  feature matrix: {best_method_ridge}")

In [ ]:
# mat_sparse_best = (feature_matrices[best_method_ridge]
#                    if sparse.issparse(feature_matrices[best_method_ridge])
#                    else sparse.csr_matrix(feature_matrices[best_method_ridge]))  

# desc_df_sparse_best = pd.DataFrame.sparse.from_spmatrix(
#     mat_sparse_best, index=describe_matrix.index
# )

In [ ]:
genome_csr_cosine = normalize(sparse.csr_matrix(feature_matrices[best_method_cosine]), axis=1, copy=False)
save_genome_csr(genome_csr_cosine)     

In [ ]:
# Lưu và load lại hồ sơ user cho cosine/ridge tốt nhất để dùng lại (tránh train lại nhiều lần)
profiles_cosine_best = load_profiles(f"{best_method_cosine}_cosine")
if profiles_cosine_best is None:
    profiles_cosine_best = build_user_profiles_sim(train_df)
    save_profiles(profiles_cosine_best, f"{best_method_cosine}_cosine")

# rmse_val_c_best, mae_val_c_best = eval_cosine(val_df, profiles_cosine_best)
# rmse_test_c_best, mae_test_c_best = eval_cosine(test_df, profiles_cosine_best)
# print(f"Cosine ({best_method_cosine})  Val RMSE={rmse_val_c_best:.4f} | Test RMSE={rmse_test_c_best:.4f}")

In [ ]:
# profiles_ridge_best = load_profiles(f"{best_method_ridge}_ridge")
# if profiles_ridge_best is None:
#     profiles_ridge_best = build_user_profiles_ridge(train_df, desc_df_sparse_best)
#     save_profiles(profiles_ridge_best, f"{best_method_ridge}_ridge")

# rmse_val_r_best, mae_val_r_best = eval_ridge(val_df, profiles_ridge_best, desc_df_sparse_best)
# rmse_test_r_best, mae_test_r_best = eval_ridge(test_df, profiles_ridge_best, desc_df_sparse_best)
# print(f"Ridge  ({best_method_ridge}) Val RMSE={rmse_val_r_best:.4f} | Test RMSE={rmse_test_r_best:.4f}")

### **B. SVD (best model with best params) Content-Based**

In [27]:
import os
import pickle
import joblib
import numpy as np
import pandas as pd
import math
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse


In [35]:
def hybrid_predict(user_id, movie_ids, user_idx, movie_idx_list, cb_profiles, cb_descriptions, funk_model, rating_min=0.5,  rating_max=5.0, alpha=0.5):
    # Content-Based: predict bằng Ridge từ userId, movieId
    pred_cb = predict_rating_ridge(user_id, movie_ids, cb_profiles, cb_descriptions, normalize_rating=False)
    if len(pred_cb) == 0:
        return np.array([])

    # Collaborative: predict bằng FunkSVD từ user_idx, movie_idx
    pred_cf = []
    for m_idx in movie_idx_list:
        if m_idx >= 0:
            pred = funk_model.predict(user_idx, m_idx)
        else:
            pred = rating_min  # fallback
        pred_cf.append(pred)

    # Kết hợp hybrid
    pred_cb = np.clip(pred_cb, rating_min, rating_max)
    pred_cf = np.clip(pred_cf, rating_min, rating_max)

    pred_hybrid = alpha * pred_cb + (1 - alpha) * np.array(pred_cf)
    return np.clip(pred_hybrid, rating_min, rating_max)


# Đánh giá RMSE / MAE cho hybrid model
def evaluate_hybrid(ratings_df, cb_profiles, cb_descriptions, funk_model, alpha=0.5, name="Val"):
    y_true, y_pred = [], []
    for uid in tqdm(ratings_df['userId'].unique(), desc=f"[{name} α={alpha:.2f}]"):
        user_df = ratings_df[ratings_df['userId'] == uid]
        movie_ids = user_df['movieId'].values
        true_ratings = user_df['rating'].values
        user_idx = user_df['user_idx'].iloc[0]
        movie_idx_list = user_df['movie_idx'].values

        pred = hybrid_predict(uid, movie_ids, user_idx, movie_idx_list, cb_profiles, cb_descriptions, funk_model, alpha=alpha)

        if len(pred) == len(true_ratings):
            y_true.extend(true_ratings)
            y_pred.extend(pred)

    if len(y_true) == 0:
        return np.nan, np.nan

    return np.sqrt(mean_squared_error(y_true, y_pred)), mean_absolute_error(y_true, y_pred)



In [29]:
funksvd_model = SurpriseFunkSVD().load("best_model_funksvd.pkl")
cb_profiles = load_profiles("entropy_ridge")

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Ridge from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [30]:
genome_csr = load_genome_csr()

# Biến cb_descriptions dùng cho Ridge Regression
cb_descriptions = pd.DataFrame(
    data=genome_csr.toarray(),
    index=describe_matrix.index
)

In [36]:
alphas = np.arange(0.0, 1.1, 0.1)
results = []

# 1. Baseline: alpha = 0.5
alpha_baseline = 0.5
rmse_val_base, mae_val_base = evaluate_hybrid(val_df, cb_profiles, cb_descriptions, funksvd_model, alpha=alpha_baseline, name="Val")
rmse_test_base, mae_test_base = evaluate_hybrid(test_df, cb_profiles, cb_descriptions, funksvd_model, alpha=alpha_baseline, name="Test")

print(f"[Baseline α={alpha_baseline:.1f}] Val RMSE: {rmse_val_base:.4f} | MAE: {mae_val_base:.4f} || Test RMSE: {rmse_test_base:.4f} | MAE: {mae_test_base:.4f}")

[Test α=0.50]: 100%|██████████| 162541/162541 [11:18<00:00, 239.52it/s]


[Baseline α=0.5] Val RMSE: 0.7615 | MAE: 0.5758 || Test RMSE: 0.7610 | MAE: 0.5755


In [37]:
# 2. Grid search alpha trên val
for alpha in alphas:
    rmse_val, mae_val = evaluate_hybrid(val_df, cb_profiles, cb_descriptions, funksvd_model, alpha=alpha)
    results.append({
        "alpha": alpha,
        "val_rmse": rmse_val,
        "val_mae": mae_val
    })
    print(f"[GridSearch α={alpha:.1f}] Val RMSE: {rmse_val:.4f} | MAE: {mae_val:.4f}")

[Val α=0.00]: 100%|██████████| 162541/162541 [10:43<00:00, 252.71it/s]


[GridSearch α=0.0] Val RMSE: 0.7731 | MAE: 0.5825


[Val α=0.10]: 100%|██████████| 162541/162541 [10:48<00:00, 250.70it/s]


[GridSearch α=0.1] Val RMSE: 0.7675 | MAE: 0.5788


[Val α=0.20]: 100%|██████████| 162541/162541 [10:29<00:00, 258.23it/s]


...

[GridSearch α=0.8] Val RMSE: 0.7742 | MAE: 0.5861


[Val α=0.90]: 100%|██████████| 162541/162541 [10:34<00:00, 256.12it/s]


[GridSearch α=0.9] Val RMSE: 0.7816 | MAE: 0.5918


[Val α=1.00]: 100%|██████████| 162541/162541 [10:22<00:00, 261.25it/s]


[GridSearch α=1.0] Val RMSE: 0.7906 | MAE: 0.5986


In [40]:
# 3. Tìm best alpha dựa trên val RMSE
df_results = pd.DataFrame(results)
best_alpha = df_results.loc[df_results['val_rmse'].idxmin(), 'alpha']

rmse_test_best, mae_test_best = evaluate_hybrid(test_df, cb_profiles, cb_descriptions, funksvd_model, alpha=best_alpha, name="Test")
print(f"[Final Test α={best_alpha:.2f}] Test RMSE: {rmse_test_best:.4f} | MAE: {mae_test_best:.4f}")

[Test α=0.40]: 100%|██████████| 162541/162541 [11:27<00:00, 236.49it/s]


[Final Test α=0.40] Test RMSE: 0.7600 | MAE: 0.5744


In [42]:
def hybrid_topk_for_user(user_id, ratings_df, movies_df, cb_profiles, cb_descriptions, funk_model, alpha=0.5, top_k=10):
    # Lấy phim đã xem
    seen = ratings_df[ratings_df['userId'] == user_id]['movieId'].values
    candidates = movies_df[~movies_df['movieId'].isin(seen)]

    if len(candidates) == 0:
        return pd.DataFrame(columns=["movieId", "predicted_rating"])

    movie_ids = candidates['movieId'].values
    movie_idx_list = candidates['movie_idx'].values
    try:
        user_idx = ratings_df[ratings_df['userId'] == user_id]['user_idx'].iloc[0]
    except IndexError:
        return pd.DataFrame(columns=["movieId", "predicted_rating"])

    preds = hybrid_predict(user_id, movie_ids, user_idx, movie_idx_list, cb_profiles, cb_descriptions, funk_model, alpha=alpha)

    if len(preds) == 0:
        return pd.DataFrame(columns=["movieId", "predicted_rating"])

    topk = candidates.copy()
    topk['predicted_rating'] = preds
    return topk.sort_values(by='predicted_rating', ascending=False).head(top_k)


In [60]:
# TMDB API setup
TMDB_API_KEY = "2307c3e2979636e48540fc3c2c7dbaec"

# Lấy đường dẫn poster từ TMDB API bằng tmdb_id.
def fetch_poster(tmdb_id, api_key):
    if pd.isna(tmdb_id):
        return None
    url = f"https://api.themoviedb.org/3/movie/{int(tmdb_id)}?api_key={api_key}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            poster_path = data.get("poster_path")
            if poster_path:
                return f"https://image.tmdb.org/t/p/w200{poster_path}"
    except:
        pass
    return None

# Thêm cột 'poster_url' vào DataFrame dựa trên movieId và links.csv.
def add_poster_column(df, links_df, api_key):
    df = df.merge(links_df[['movieId', 'tmdbId']], on='movieId', how='left')
    df['poster_url'] = df['tmdbId'].apply(lambda x: fetch_poster(x, api_key))
    return df.drop(columns=['tmdbId'])

# Hiển thị poster
def display_with_posters(df):
    html = "<table><tr><th>Movie ID</th><th>Title</th><th>Genres</th><th>Predicted Rating</th><th>Poster</th></tr>"
    for _, row in df.iterrows():
        poster_img = (
            f"<img src='{row['poster_url']}' width='200' height='280'>" 
            if row['poster_url'] else "N/A"
        )
        html += (
            f"<tr>"
            f"<td>{row['movieId']}</td>"
            f"<td>{row['title']}</td>"
            f"<td>{row['genres']}</td>"
            f"<td>{row['predicted_rating']:.2f}</td>"
            f"<td>{poster_img}</td>"
            f"</tr>"
        )
    html += "</table>"
    return HTML(html)

In [70]:
# Gộp movie_idx từ cả 3 tập
movie_idx_map = pd.concat([train_df, val_df, test_df])[['movieId', 'movie_idx']].drop_duplicates()
movies_df = movies.merge(movie_idx_map, on='movieId', how='left')
topk_df['movieId'] = topk_df['movieId'].astype(int)
links['movieId'] = links['movieId'].astype(int)

In [71]:
user_id = 95133  # ID người dùng bạn muốn gợi ý
topk_df = hybrid_topk_for_user(
    user_id=user_id,
    ratings_df=val_df,
    movies_df=movies_df,
    cb_profiles=cb_profiles,
    cb_descriptions=cb_descriptions,
    funk_model=funksvd_model,
    alpha=0.4,
    top_k=10
)

In [72]:
topk_df = add_poster_column(topk_df, links, TMDB_API_KEY)
display_with_posters(topk_df)


Movie ID,Title,Genres,Predicted Rating,Poster
2324,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama|Romance|War,4.64,
318,"Shawshank Redemption, The (1994)",Crime|Drama,4.51,
120807,John Mulaney: New In Town (2012),Comedy,4.49,
2858,American Beauty (1999),Drama|Romance,4.46,
134853,Inside Out (2015),Adventure|Animation|Children|Comedy|Drama|Fantasy,4.40,
65188,Dear Zachary: A Letter to a Son About His Father (2008),Documentary,4.40,
163134,Your Name. (2016),Animation|Drama|Fantasy|Romance,4.40,
1704,Good Will Hunting (1997),Drama|Romance,4.39,
527,Schindler's List (1993),Drama|War,4.35,
60069,WALL·E (2008),Adventure|Animation|Children|Romance|Sci-Fi,4.34,
